In [ ]:
import analytics.api as ga
import analytics.sheets_api as sheets
import analytics.sheets_elements as elements
import pandas as pd
import gspread
from constants import *

In [ ]:
%env HCA_ANALYTICS_REPORTING_CLIENT_SECRET_PATH=../../do_not_commit_ga4_credentials.json

In [ ]:
ga_authentication, drive_authentication, sheets_authentication = ga.authenticate(
    SECRET_NAME,
    ga.ga4_service_params,
    ga.drive_service_params,
    ga.sheets_service_params,
    port=OAUTH_PORT
)

date_string = f"{START_DATE_CURRENT} - {END_DATE_CURRENT}"

default_params = {
    "service_system": ga_authentication,
    "start_date": START_DATE_CURRENT,
    "end_date": END_DATE_CURRENT,
    "property": HCA_PORTAL_ID,
}

hca_portal_params = {
    **default_params,
    "base_dimension_filter": HCA_BROWSER_EXCLUDE_FILTER,
}

hca_combined_all_time_params = {
    **default_params,
    "start_date": ANALYTICS_START,
    "end_date": END_DATE_CURRENT,
}

hca_portal_all_time_params = {
    **hca_portal_params,
    "start_date": ANALYTICS_START,
}

In [ ]:
df_monthly_summary_portal = elements.get_page_views_over_time_df(hca_portal_all_time_params)
df_monthly_summary_combined = elements.get_page_views_over_time_df(hca_combined_all_time_params, additional_data_path=HISTORIC_UA_DATA_PATH, additional_data_behavior=elements.ADDITIONAL_DATA_BEHAVIOR.ADD)
df_outbound = elements.get_outbound_links_change(hca_portal_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)
df_pageviews = elements.get_page_views_change(hca_portal_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)

In [ ]:
dict_spreadsheet = {
    "Portal Summary": df_monthly_summary_portal,
    "Combined Summary": df_monthly_summary_combined,
    "Page Views": df_pageviews,
    "Outbound Links": df_outbound,
}

summary_column_formatting = {
    "Month": sheets.COLUMN_FORMAT_OPTIONS.YEAR_MONTH_DATE,
    "Users Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
    "Total Pageviews Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
}

summary_worksheet_formatting = {
    "extra_columns": 1,
    "extra_columns_width": 2000,
}

sheet = sheets.create_sheet_in_folder(
    drive_authentication,
    SHEET_NAME,
    PARENT_FOLDER_NAME,
    override_behavior=sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE
)

sheets.fill_spreadsheet_with_df_dict(
    sheet,
    dict_spreadsheet,
    sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE,
    column_formatting_options={
        "Portal Summary": summary_column_formatting,
        "Combined Summary": summary_column_formatting,
        "Page Views": {
            "Total Users Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Views Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Outbound Links": {
            "Total Clicks Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Users Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        }
    },
    sheet_formatting_options={
        "Portal Summary": summary_worksheet_formatting,
        "Combined Summary": summary_worksheet_formatting
    }
)
for df, worksheet_name in zip((df_monthly_summary_portal, df_monthly_summary_combined), ("Portal Summary", "Combined Summary")):
    worksheet = sheet.worksheet(worksheet_name)
    date_range = sheets.WorksheetRange(
        worksheet, 
        gspread.cell.Cell(1, 1), 
        gspread.cell.Cell(df.index.size + 1, 2)
    )
    users_range = sheets.WorksheetRange(
        worksheet, 
        gspread.cell.Cell(1, 2), 
        gspread.cell.Cell(df.index.size + 1, 3)
    )
    pageviews_range = sheets.WorksheetRange(
        worksheet, 
        gspread.cell.Cell(1, 3), 
        gspread.cell.Cell(df.index.size + 1, 4)
    )
    sheets.add_chart_to_sheet(
        sheets_authentication,
        sheet,
        worksheet,
        sheets.CHART_TYPES.LINE,
        date_range,
        [users_range, pageviews_range],
        chart_position=gspread.cell.Cell(1, 6),
        chart_position_offset_x=75,
        chart_position_offset_y=25,
        title="Pageviews and Users Over Time"
    )